# Разработка системы распознавания предметов интерьера в потоковом видео

## Выявление частоты классов в датасете

In [3]:
%matplotlib inline

#System
import os
import shutil
import random

#Base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from skimage.io import imread, imshow, imsave


#Keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPool2D, UpSampling2D
from keras.layers import Dropout,BatchNormalization, Concatenate
from keras.optimizers import Adam, SGD, RMSprop

#Preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

#Models
from keras.applications.vgg16 import VGG16

#GPU
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto() 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config) 
K.set_session(sess)

#Seed
seed = 99
np.random.seed(seed)
tf.set_random_seed(seed)
random.seed(seed)

In [37]:
def get_format(file_name):
    name_split = file_name.split("_")
    file_format = name_split[0:-1] + name_split[-1].split(".")
    ade, train_or_val, name = file_format[:3]
    extension = file_format[-1]
    
    description = 0
    parts_num = 0
    if extension == 'jpg':
        description = 'img'
    elif extension == 'txt':
        description = 'text'
    else: #png
        description = file_format[3]
        if file_format[4].isdigit():
            parts_num = int(file_format[4])
    return [name, description, train_or_val, parts_num]

In [38]:
class SegEncoder:
    def __init__(self, objectnames_path):
        with open(objectnames_path, 'r') as f:
            self.obj_names = f.readline().split("\t")
        self.obj_names = list(map(lambda name: name.strip("\'"), self.obj_names))
    
    def get_class_index(self, pixel):
        R = pixel[0]
        G = pixel[1]
        B = pixel[2]
        return int(R / 10 * 256 + G) - 1
    
    def get_class(self, pixel):
        return self.obj_names[self.get_class_index(pixel)]
    
segEncoder = SegEncoder('data/ADE20K_filtred/objectnames.txt')

In [40]:
index_old_to_new_dict = dict()
with open('data/ADE20K_filtred/class_old_new_filt.csv', 'r') as f:
    for line in f:
        line_arr = line.strip().split(";")
        index_old_to_new_dict[int(line_arr[1])] = int(line_arr[2])

In [41]:
index_old_to_new_dict

{0: 0,
 470: 1,
 1734: 2,
 349: 3,
 2683: 4,
 1394: 5,
 773: 6,
 164: 7,
 56: 8,
 1830: 9,
 2472: 10,
 2387: 11,
 3086: 12,
 723: 13,
 2678: 14,
 570: 15,
 2585: 16,
 490: 17,
 2732: 18,
 529: 19,
 2597: 20,
 590: 21,
 2729: 22,
 2792: 23,
 154: 24,
 2095: 25,
 2984: 26,
 1550: 27,
 238: 28,
 1582: 29,
 746: 30,
 1707: 31}

Преобразуем его для перекрытия всех индексов

In [72]:
for index in range(len(segEncoder.obj_names)):
    if not index in index_old_to_new_dict:
        index_old_to_new_dict[index] = 0

In [42]:
def get_encoded_seg(seg):
    new_seg = seg[:, :, 0].copy()
    for i in range(new_seg.shape[0]):
        for j in range(new_seg.shape[1]):
            pixel = seg[i, j]
            class_index = segEncoder.get_class_index(pixel)
            if class_index in index_old_to_new_dict:
                new_seg[i][j] = np.uint8(index_old_to_new_dict[class_index])
            else:
                new_seg[i][j] = 0

    return new_seg

In [60]:
seg = imread("data/ADE20K_filtred/images/train/c/computer_room/" + "ADE_train_00005939_seg.png")

In [152]:
def get_encoded_seg(seg):
    new_seg = seg[:, :, 0].copy()
    for i in range(new_seg.shape[0]):
        for j in range(new_seg.shape[1]):
            pixel = seg[i, j]
            class_index = segEncoder.get_class_index(pixel)
            new_seg[i][j] = class_index

    return new_seg

In [153]:
get_encoded_seg(seg)

Array([[255, 255, 255, ..., 161, 255, 255],
       [255, 255, 161, ..., 161, 255, 255],
       [255, 255, 161, ..., 161, 255, 255],
       ...,
       [255, 255, 207, ..., 207, 207, 255],
       [255, 255, 207, ..., 207, 207, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [270]:
index_old_to_new_dict.k

{0: 0,
 470: 1,
 1734: 2,
 349: 3,
 2683: 4,
 1394: 5,
 773: 6,
 164: 7,
 56: 8,
 1830: 9,
 2472: 10,
 2387: 11,
 3086: 12,
 723: 13,
 2678: 14,
 570: 15,
 2585: 16,
 490: 17,
 2732: 18,
 529: 19,
 2597: 20,
 590: 21,
 2729: 22,
 2792: 23,
 154: 24,
 2095: 25,
 2984: 26,
 1550: 27,
 238: 28,
 1582: 29,
 746: 30,
 1707: 31,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87:

In [302]:
def new_get_encoded_seg(seg):
    R = seg[:, :, 0].copy()
    G = seg[:, :, 1].copy()
    B = seg[:, :, 2].copy()

    new_seg = (R / 10 * 256 + G).astype(np.uint16)
    new_seg[new_seg != 0] -= 1
    
    old_list = sorted(index_old_to_new_dict.keys())
    for old in old_list:
        new_seg[new_seg == old] = index_old_to_new_dict[old]

    return new_seg

In [303]:
%%time 
#старая версия
new_seg = get_encoded_seg(seg)

CPU times: user 221 ms, sys: 0 ns, total: 221 ms
Wall time: 220 ms


In [304]:
%%time 
#новая версия
new_new_seg = new_get_encoded_seg(seg)

CPU times: user 215 ms, sys: 0 ns, total: 215 ms
Wall time: 214 ms


Как мы видим программа значительно ускорилась

### Генерирование новой обучающей выборки

In [112]:
def get_format(file_name):
    name_split = file_name.split("_")
    file_format = name_split[0:-1] + name_split[-1].split(".")
    ade, train_or_val, name = file_format[:3]
    extension = file_format[-1]
    
    description = 0
    parts_num = 0
    if extension == 'jpg':
        description = 'img'
    elif extension == 'txt':
        description = 'text'
    else: #png
        description = file_format[3]
        if file_format[4].isdigit():
            parts_num = int(file_format[4])
    return [name, description, train_or_val, parts_num]

In [308]:
%%time

dir_path = 'data/ADE20K_filtred/images/train/'

img_dst_train = 'data/ADE20K_formated/train/img/img/'
mask_dst_train = 'data/ADE20K_formated/train/mask/mask/'

img_dst_val = 'data/ADE20K_formated/val/img/img/'
mask_dst_val = 'data/ADE20K_formated/val/mask/mask/'

s = 0
for path, dirs, files in os.walk(dir_path):
        if files:
            for file in files:
                name, description, train_or_val, parts_num = get_format(file)
                
                img_dst = img_dst_train
                mask_dst = mask_dst_train
                if train_or_val == 'val':
                    img_dst = img_dst_val
                    mask_dst = mask_dst_val
                
                if description == 'img':
                    img_path = path + '/' + file
                    shutil.copyfile(img_path, img_dst + name + '.jpg')
                    s += 1
                    
                    if s % 100 == 0:
                        print(s)
                    
                elif description == 'seg':
                    seg_path = path + '/' + file
                    seg = imread(seg_path)
                    new_seg = get_encoded_seg(seg)
                    imsave(mask_dst + name + '.png', new_seg)
                
print("Done")

/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016716.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016714.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/val/mask/mask/00001769.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016711.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016713.png is a low contrast image
  warn('%s is a low contrast image' %

/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016282.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/val/mask/mask/00000750.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016292.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016294.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016284.png is a low contrast image
  warn('%s is a low contrast image' %

/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016818.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/val/mask/mask/00001775.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/val/mask/mask/00001773.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016825.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016820.png is a low contrast image
  warn('%s is a low contrast image' % f

100


/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016791.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016789.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016808.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/val/mask/mask/00000775.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00016835.png is a low contrast image
  warn('%s is a low contrast image' %

...

7100


/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00000884.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00000695.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00011330.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00011366.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/val/mask/mask/00000520.png is a low contrast image
  warn('%s is a low contrast image' %

/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00011365.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00011359.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00011483.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00000784.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00000651.png is a low contrast image
  warn('%s is a low contrast image'

Done
CPU times: user 2h 4min 53s, sys: 1min 56s, total: 2h 6min 49s
Wall time: 2h 10min 43s


/home/rv/anaconda3/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: data/ADE20K_formated/train/mask/mask/00011224.png is a low contrast image
  warn('%s is a low contrast image' % fname)
